In [1]:
# import the necessary packages
from imageSearch.centroidtracker import CentroidTracker
from imageSearch.trackableobject import TrackableObject
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2


 imutils.video  will help us to work with a webcam and to calculate the estimated Frames Per Second (FPS) throughput rate   
 dlib  library will be used for its correlation tracker implementation.

In [2]:
#IF we run into cmd Script way
# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-p", "--prototxt", required=True,
# 	help="path to Caffe 'deploy' prototxt file")
# ap.add_argument("-m", "--model", required=True,
# 	help="path to Caffe pre-trained model")
# ap.add_argument("-i", "--input", type=str,
# 	help="path to optional input video file")
# ap.add_argument("-o", "--output", type=str,
# 	help="path to optional output video file")
# ap.add_argument("-c", "--confidence", type=float, default=0.4,
# 	help="minimum probability to filter weak detections")
# ap.add_argument("-s", "--skip-frames", type=int, default=30,
# 	help="# of skip frames between detections")
# args = vars(ap.parse_args())

In [3]:
# initialize the list of class labels MobileNet SSD was trained to
# detect
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]
srcPath = r'videos/example_01.mp4'
protoTxtPath = r'YOLO_DB/MobileNetSSD_deploy.prototxt'
modelPath = r'YOLO_DB/MobileNetSSD_deploy.caffemodel'

In [4]:
print("[INFO] init model...")
net = cv2.dnn.readNetFromCaffe(protoTxtPath, modelPath)

[INFO] init model...


In [5]:
cam = cv2.VideoCapture(srcPath)
writer = None
width = None
height = None

In [6]:
ct = CentroidTracker(maxDisappeared=50, maxDistance=50)
trackers = []
trackableObjects= {} # map each unique object ID to a TrackableObject

In [7]:
# with the total number of objects that have moved either up or down
totalFrames = 0
totalDown = 0
totalUp = 0

In [8]:
# start the frames per second throughput estimator
# fps = FPS().start()
#fps of video
fps = int(cam.get(cv2.CAP_PROP_FPS))
threshold = .4

In [10]:
while True:
    _, img = cam.read()
    
    if img is None:
        break
    
    #resize frame
    img = imutils.resize(img, width=500)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    if height is None or width is None:
        (height, width,_) = img.shape

    status = "Waiting"
    rects = []
    
    # check to see if we should run a more computationally expensive
    # object detection method to aid our tracker
    if totalFrames % fps == 0:
        # set the status and initialize our new set of object trackers
        status = "Detecting"
        trackers = []
        
        # detecting people using the SSD
        blob = cv2.dnn.blobFromImage(img, 0.007843, (width, height), 127.5)
        net.setInput(blob)
        detections = net.forward()
        
        
        #Detected obj
#         print(detections.shape[2])
        for i in np.arange(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            
            if confidence > threshold :
                index = int(detections[0, 0, i, 1])
                
            # if the class label is not a person, ignore it
                if CLASSES[index] != "person":
                    continue
            
                #get Bounding box
                box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
                (startX, startY, endX, endY) = box.astype("int")
                
                tracker = dlib.correlation_tracker()
                rect = dlib.rectangle(startX, startY, endX, endY)
                tracker.start_track(rgb, rect)
                
                trackers.append(tracker)
                    
    
    else:
        for tracker in trackers:
            status = "Tracking.."
            
            # update the tracker and grab the updated position
            tracker.update(rgb)
            pos = tracker.get_position()
            
            # unpack the position object
            startX = int(pos.left())
            startY = int(pos.top())
            endX = int(pos.right())
            endY = int(pos.bottom())
            # add the bounding box coordinates to the rectangles list
            rects.append((startX, startY, endX, endY))

            
    # draw a horizontal line in the center of the frame -- once an
    # object crosses this line we will determine whether they were
    # moving 'up' or 'down'
    cv2.line(img, (0, height // 2), (width, height // 2), (0, 255, 255), 2)
    
    # use the centroid tracker to associate the (1) old object
    # centroids with (2) the newly computed object centroids
    objects = ct.update(rects)
            
#     if len(objects)>0:
#         print(len(objects))
        
    for (objectID, centroid) in objects.items():
        # check to see if a trackable object exists for the current
        # object ID
        to = trackableObjects.get(objectID, None)
    
        # if there is no existing trackable object, create one
        if to is None:
            to = TrackableObject(objectID, centroid)

        else:
            # the difference between the y-coordinate of the *current*
            # centroid and the mean of *previous* centroids will tell
            # us in which direction the object is moving (negative for
            # 'up' and positive for 'down')
            y = [c[1] for c in to.centroids]
            direction = centroid[1] - np.mean(y)
            to.centroids.append(centroid)
            
            # check to see if the object has been counted or not
            if not to.counted:
				# if the direction is negative (indicating the object
				# is moving up) AND the centroid is above the center
				# line, count the object
                if direction < 0 and centroid[1] < height // 2:
                    totalUp += 1
                    to.counted = True
                    
                # if the direction is positive (indicating the object
				# is moving down) AND the centroid is below the
				# center line, count the object
                elif direction > 0 and centroid[1] > height // 2:
                    totalDown += 1
                    to.counted = True
		# store the trackable object in our dictionary
        trackableObjects[objectID] = to
        
        #information on the frame for visualization:
        text = "ID {}".format(objectID)
        cv2.putText(img, text, (centroid[0] - 10, centroid[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.circle(img, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

    info = [("Up", totalUp),("Down", totalDown),("Status", status),]
        
    for (i, (k, v)) in enumerate(info):
        text = "{}: {}".format(k, v)
        cv2.putText(img, text, (10, height - ((i * 20) + 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

#     if len(trackers) >0:
#         print(len(trackers))
       
       
    cv2.imshow("video", img)
    key = cv2.waitKey(1000// fps) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
        
    totalFrames+=1
    
cv2.destroyAllWindows()

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
